In [1]:
import requests
import json
import numpy as np
import pandas as pd
import time

In [2]:
# application_products = {'khame':[],'shir':[],'mast':[],'doogh':[],'panir':[],'kashk':[],'deser':[]}
application_products = []

In [3]:
# application_information = {'khame':{},'shir':{},'mast':{},'doogh':{},'panir':{},'kashk':{},'deser':{}}
application_information = {}

In [4]:
products_base_url = 'https://api.digikala.com/v1/categories/dairy/search/?page=1'

In [5]:
products_base_dict = '---'

while (products_base_dict=='---'):
    try:
        products_base_dict = json.loads(requests.get(products_base_url,timeout=5).text)
    except:
        time.sleep(0.1)

In [6]:
products_base_data = products_base_dict['data']

In [7]:
application_information['number_of_products'] = products_base_data['pager']['total_items']
application_information['number_of_pages'] = products_base_data['pager']['total_pages']
application_information['category_id'] = products_base_data['category']['id']
application_information['category_persian_title'] = products_base_data['category']['title_fa']
application_information['category_english_title'] = products_base_data['category']['code']
application_information['brands'] = [{'brand_id':brand['id'],'brand_persian_title':brand['title_fa'],'brand_english_title':brand['code']} for brand in products_base_data['filters']['brands']['options']]

In [8]:
def extract_type(x):
    
    if('پنیر' in x.split(' ')):
        return 'پنیر'
    elif('شیر' in x.split(' ')):
        return 'شیر'
    elif('ماست' in x.split(' ')):
        return 'ماست'
    elif('دوغ' in x.split(' ')):
        return 'دوغ'
    elif('کره' in x.split(' ')):
        return 'کره'
    elif('خامه' in x.split(' ')):
        return 'خامه'
    elif('کشک' in x.split(' ')):
        return 'کشک'
    elif('بستنی' in x.split(' ')):
        return 'بستنی'
    else:
        return 'سایر'

In [9]:
def status_translate(x):
    if(x=='marketable'):
        return 'قابل فروش'
    if(x=='coming_soon'):
        return 'به زودی'
    if(x=='in_supply'):
        return 'در حال تامین'
    if(x=='stop_production'):
        return 'توقف تولید' 

In [10]:
def offer_translate(x):
    if(x=='none'):
        return 'معمولی'
    if(x=='special-offer'):
        return 'فروش ویژه'
    if(x=='incredible'):
        return 'پیشنهاد شگفت انگیز'

In [11]:
def rial_be_toman(x):
    try:
        return int(x/10)
    except:
        return '---'

In [12]:
lang_converter = {'کیفیت و کارایی': 'Quality and efficiency' ,'قیمت و ارزش خرید': 'Price and purchase value' ,'ابعاد یا سایز': 'Dimensions or size' ,'تاریخ تولید یا مصرف': 'Date of production or consumption' ,'شباهت یا مغایرت': 'Similarity or contrast' ,'طعم یا رایحه': 'taste or aroma'}

In [13]:
for product in products_base_data['products']:

    product_dict = {}

    try:
        product_dict['product_id'] = product['id']
    except:
        product_dict['product_id'] = '---'
    try:
        product_dict['product_persian_title'] = product['title_fa']
    except:
        product_dict['product_persian_title'] = '---'
    try:
        product_dict['product_english_title'] = product['title_en']
    except:
        product_dict['product_english_title'] = '---'
    try:
        product_dict['product_star_rate'] = product['rating']['rate']
    except:
        product_dict['product_star_rate'] = '---'
    try:
        product_dict['product_star_number_of_scorer'] = product['rating']['count']
    except:
        product_dict['product_star_number_of_scorer'] = '---'
    try:
        product_dict['product_status'] = status_translate(
            product['status'])
    except:
        product_dict['product_status'] = '---'
    try:
        product_dict['product_brand'] = product['data_layer']['brand']
    except:
        product_dict['product_brand'] = '---'
    try:
        product_dict['product_offer'] = offer_translate(
            product['data_layer']['dimension7'])
    except:
        product_dict['product_offer'] = '---'
    try:
        product_dict['product_min_price_last_month'] = rial_be_toman(
            product['properties']['min_price_in_last_month'])
    except:
        product_dict['product_min_price_last_month'] = '---'
    try:
        product_dict['product_satisfy_rate'] = product['default_variant']['statistics']['total_rate']
    except:
        product_dict['product_satisfy_rate'] = '---'
    try:
        product_dict['product_satisfy_number_of_scorer'] = product['default_variant']['statistics']['total_count']
    except:
        product_dict['product_satisfy_number_of_scorer'] = '---'
    try:
        product_dict['product_digiplus_cashback'] = rial_be_toman(
            product['default_variant']['digiplus']['cash_back'])
    except:
        product_dict['product_digiplus_cashback'] = '---'
    try:
        product_dict['product_digiclub_point'] = product['default_variant']['digiclub']['point']
    except:
        product_dict['product_digiclub_point'] = '---'
    try:
        product_dict['product_selling_price'] = rial_be_toman(
            product['default_variant']['price']['selling_price'])
    except:
        product_dict['product_selling_price'] = '---'
    try:
        product_dict['product_rrp_price'] = rial_be_toman(
            product['default_variant']['price']['rrp_price'])
    except:
        product_dict['product_rrp_price'] = '---'
    try:
        product_dict['product_order_limit'] = product['default_variant']['price']['order_limit']
    except:
        product_dict['product_order_limit'] = '---'
    try:
        product_dict['product_discount_percent'] = product['default_variant']['price']['discount_percent']
    except:
        product_dict['product_discount_percent'] = '---'
    try:
        product_dict['product_group'] = extract_type(product['title_fa'])
    except:
        product_dict['product_group'] = '---'
        
    print('done')
        
    comments_base_url = 'https://api.digikala.com/v1/product/{}/comments/'.format(product_dict['product_id'])  
    
    comments_base_data = '---'

    while (comments_base_data=='---'):
        try:
            comments_base_data = json.loads(requests.get(comments_base_url,timeout=5).text)['data']
            print('done2')
        except:
            time.sleep(0.1)
    print('done3')
            
    try:
        product_dict['product_total_comments'] = comments_base_data['pager']['total_items']
    except:
        product_dict['product_total_comments'] = '---'
        
    print('done4')
    
    for item in comments_base_data['intent_data']: 
        
        try:
            product_dict[lang_converter[item['title']]+'Number_Of_Comments'] = item['number_of_comments']
        except:
            product_dict[lang_converter[item['title']]+'Number_Of_Comments'] = '---'
            
        try:
            product_dict[lang_converter[item['title']]+'positive'] = item['tag_percentage']['positive']
        except:
            product_dict[lang_converter[item['title']]+'positive'] = '---'
            
        try:
            product_dict[lang_converter[item['title']]+'negative'] = item['tag_percentage']['negative']
        except:
            product_dict[lang_converter[item['title']]+'negative'] = '---'
            
        try:
            product_dict[lang_converter[item['title']]+'neutral'] = item['tag_percentage']['neutral']
        except:
            product_dict[lang_converter[item['title']]+'neutral'] = '---'
            
        print('done5')
        

    application_products.append(product_dict)



done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
done5
done
done2
done3
done4
done5
done5
done5
done5
done5
don

In [14]:
for page_number in range(2, (application_information['number_of_pages'])+1):

    url = 'https://api.digikala.com/v1/categories/dairy/search/?page={}'.format(
        page_number)

    products_dict = '---'

    while (products_dict == '---'):
        try:
            products_dict = json.loads(requests.get(url, timeout=5).text)
        except:
            time.sleep(0.1)

    # print(application_information['number_of_pages'])

    # print(page_number)

    if(page_number<=100):


        products_data = products_dict['data']

        for product in products_data['products']:

            product_dict = {}

            try:
                product_dict['product_id'] = product['id']
            except:
                product_dict['product_id'] = '---'
            try:
                product_dict['product_persian_title'] = product['title_fa']
            except:
                product_dict['product_persian_title'] = '---'
            try:
                product_dict['product_english_title'] = product['title_en']
            except:
                product_dict['product_english_title'] = '---'
            try:
                product_dict['product_star_rate'] = product['rating']['rate']
            except:
                product_dict['product_star_rate'] = '---'
            try:
                product_dict['product_star_number_of_scorer'] = product['rating']['count']
            except:
                product_dict['product_star_number_of_scorer'] = '---'
            try:
                product_dict['product_status'] = status_translate(
                    product['status'])
            except:
                product_dict['product_status'] = '---'
            try:
                product_dict['product_brand'] = product['data_layer']['brand']
            except:
                product_dict['product_brand'] = '---'
            try:
                product_dict['product_offer'] = offer_translate(
                    product['data_layer']['dimension7'])
            except:
                product_dict['product_offer'] = '---'
            try:
                product_dict['product_min_price_last_month'] = rial_be_toman(
                    product['properties']['min_price_in_last_month'])
            except:
                product_dict['product_min_price_last_month'] = '---'
            try:
                product_dict['product_satisfy_rate'] = product['default_variant']['statistics']['total_rate']
            except:
                product_dict['product_satisfy_rate'] = '---'
            try:
                product_dict['product_satisfy_number_of_scorer'] = product['default_variant']['statistics']['total_count']
            except:
                product_dict['product_satisfy_number_of_scorer'] = '---'
            try:
                product_dict['product_digiplus_cashback'] = rial_be_toman(
                    product['default_variant']['digiplus']['cash_back'])
            except:
                product_dict['product_digiplus_cashback'] = '---'
            try:
                product_dict['product_digiclub_point'] = product['default_variant']['digiclub']['point']
            except:
                product_dict['product_digiclub_point'] = '---'
            try:
                product_dict['product_selling_price'] = rial_be_toman(
                    product['default_variant']['price']['selling_price'])
            except:
                product_dict['product_selling_price'] = '---'
            try:
                product_dict['product_rrp_price'] = rial_be_toman(
                    product['default_variant']['price']['rrp_price'])
            except:
                product_dict['product_rrp_price'] = '---'
            try:
                product_dict['product_order_limit'] = product['default_variant']['price']['order_limit']
            except:
                product_dict['product_order_limit'] = '---'
            try:
                product_dict['product_discount_percent'] = product['default_variant']['price']['discount_percent']
            except:
                product_dict['product_discount_percent'] = '---'
            try:
                product_dict['product_group'] = extract_type(product['title_fa'])
            except:
                product_dict['product_group'] = '---'
                
            comments_base_url = 'https://api.digikala.com/v1/product/{}/comments/'.format(product_dict['product_id'])  
    
            comments_base_data = '---'

            while (comments_base_data=='---'):
                try:
                    comments_base_data = json.loads(requests.get(comments_base_url,timeout=5).text)['data']
                except:
                    time.sleep(0.1)

            try:
                product_dict['product_total_comments'] = comments_base_data['pager']['total_items']
            except:
                product_dict['product_total_comments'] = '---'

            for item in comments_base_data['intent_data']: 

                try:
                    product_dict[lang_converter[item['title']]+'Number_Of_Comments'] = item['number_of_comments']
                except:
                    product_dict[lang_converter[item['title']]+'Number_Of_Comments'] = '---'

                try:
                    product_dict[lang_converter[item['title']]+'positive'] = item['tag_percentage']['positive']
                except:
                    product_dict[lang_converter[item['title']]+'positive'] = '---'

                try:
                    product_dict[lang_converter[item['title']]+'negative'] = item['tag_percentage']['negative']
                except:
                    product_dict[lang_converter[item['title']]+'negative'] = '---'

                try:
                    product_dict[lang_converter[item['title']]+'neutral'] = item['tag_percentage']['neutral']
                except:
                    product_dict[lang_converter[item['title']]+'neutral'] = '---'

            application_products.append(product_dict)

    else:
        pass


In [15]:
# col_names = ['کد محصول','نام فارسی محصول','نام انگلیسی محصول','نرخ ستاره محصول','تعداد نفرات امتیاز دهنده ستاره محصول','وضعیت محصول','برند محصول','نوع پیشنهاد فروش محصول','کمترین قیمت در ماه گذشته محصول','نرخ رضایت محصول','تعداد نفرات ثبت رضایت محصول','مقدار جایزه دیجی پلاس محصول','مقدار امتیاز دیجی کلاب محصول','قیمت فروش محصول','قیمت اصلی محصول','حداکثر تعداد سفارش محصول','درصد تخفیف محصول','گروه محصول']

In [16]:
with pd.ExcelWriter("Products_Digi_V2_Comments.xlsx") as writer:

    df = pd.DataFrame(application_products)
#     df.columns = col_names
#     df = df.drop(['نام انگلیسی محصول'], axis=1)
    df.to_excel(writer, sheet_name='Products_Comments', index=False)
